# Validation of islanded mode equivalence

## Import and helpers
Import core packages for the toy model.

In [2]:
import logging
import numpy as np
import pandas as pd
import pypsa
import plotly.graph_objects as go
logger = logging.getLogger(__name__)

Helpers

In [3]:
def extract_capacity(n: pypsa.Network) -> pd.DataFrame:
    """Extract a tidy table of optimal capacities for all extendable assets."""
    df = n.statistics.optimal_capacity(nice_names=False).reset_index()
    df.columns = ["component", "name", "capacity"]
    return df


def extract_energy_balance(n: pypsa.Network) -> pd.DataFrame:
    """Extract a tidy table of energy balance statistics."""
    df = n.statistics.energy_balance(nice_names=False)
    df.rename("energy", inplace=True)
    return df.reset_index()


def calculate_total_costs(n: pypsa.Network, exclude_grid: bool = False) -> float:
    """Calculate total costs (capex + opex), optionally excluding AC lines and DC links."""
    costs = pd.concat([n.statistics.capex(nice_names=False), n.statistics.opex(nice_names=False)], axis=1, keys=["capex", "opex"])
    if exclude_grid:
        costs = costs[~(
            ((costs.index.get_level_values("component") == "Line") &
             (costs.index.get_level_values("carrier") == "AC")) |
            ((costs.index.get_level_values("component") == "Link") &
             (costs.index.get_level_values("carrier") == "DC"))
        )]
    return costs.fillna(0).sum().sum()


def create_bar_plot(
    pivot_df, 
    x_col, 
    y_cols, 
    labels, 
    title, 
    yaxis_title,
    height=600,
    width=800,
):
    """Create a grouped bar plot with multiple scenarios."""
    fig = go.Figure()
    for y_col, label in zip(y_cols, labels):
        fig.add_trace(go.Bar(
            x=pivot_df[x_col], 
            y=pivot_df[y_col], 
            name=label,
        ))
    fig.update_layout(
        title=title,
        xaxis_title=x_col.title(),
        yaxis_title=yaxis_title,
        barmode="group",
        hovermode="x unified",
        template="plotly_white",
        height=height,
        width=width,
    )
    return fig


def create_stacked_bar_plot(
    df, 
    scenario_col, 
    value_col, 
    carrier_col, 
    carrier_colors, 
    scenario_labels, 
    title, 
    yaxis_title,
    height=800,
    width=600,
):
    """Create a stacked bar plot with carriers colored by carrier_colors dict.
    
    Positive values are stacked upward (generation), negative values downward (demand).
    
    Parameters
    ----------
    df : pd.DataFrame
        Long-form dataframe with scenario, carrier, and value columns
    scenario_col : str
        Column name containing scenario identifiers
    value_col : str
        Column name containing values to plot
    carrier_col : str
        Column name containing carrier names
    carrier_colors : dict
        Dictionary mapping carrier names to colors
    scenario_labels : dict
        Dictionary mapping scenario identifiers to display labels
    title : str
        Plot title
    yaxis_title : str
        Y-axis label
    """
    fig = go.Figure()
    
    # Define the order of scenarios
    scenario_order = ["grid", "maxpu0", "removed", "outage_summer", "outage_winter"]
    
    # Get unique carriers
    carriers = df[carrier_col].unique()
    
    # Create a complete dataframe with all scenario-carrier combinations
    # Group by scenario and carrier to aggregate any duplicates
    df_grouped = df.groupby([scenario_col, carrier_col], as_index=False)[value_col].sum()
    
    # Separate positive and negative values
    for carrier in carriers:
        carrier_data = df_grouped[df_grouped[carrier_col] == carrier].copy()
        
        # Split into positive and negative
        positive_data = carrier_data[carrier_data[value_col] >= 0].copy()
        negative_data = carrier_data[carrier_data[value_col] < 0].copy()
        
        color = carrier_colors.get(carrier, "#cccccc")
        
        # Add positive values (generation)
        if not positive_data.empty:
            # Ensure all scenarios are present
            x_vals = []
            y_vals = []
            for scenario in scenario_order:
                x_vals.append(scenario_labels.get(scenario, scenario))
                matching = positive_data[positive_data[scenario_col] == scenario]
                if not matching.empty:
                    y_vals.append(matching[value_col].values[0])
                else:
                    y_vals.append(0)
            
            fig.add_trace(go.Bar(
                name=carrier,
                x=x_vals,
                y=y_vals,
                marker=dict(color=color),
                legendgroup=carrier,
            ))
        
        # Add negative values (demand)
        if not negative_data.empty:
            # Ensure all scenarios are present
            x_vals = []
            y_vals = []
            for scenario in scenario_order:
                x_vals.append(scenario_labels.get(scenario, scenario))
                matching = negative_data[negative_data[scenario_col] == scenario]
                if not matching.empty:
                    y_vals.append(matching[value_col].values[0])
                else:
                    y_vals.append(0)
            
            fig.add_trace(go.Bar(
                name=carrier,
                x=x_vals,
                y=y_vals,
                marker=dict(color=color),
                legendgroup=carrier,
                showlegend=False,  # Don't duplicate in legend
            ))
    
    fig.update_layout(
        title=title,
        xaxis_title="Scenario",
        yaxis_title=yaxis_title,
        barmode="relative",  # This stacks positive up and negative down
        hovermode="x unified",
        template="plotly_white",
        height=height,
        width=width,
    )
    
    return fig

## Data prep
Load the base network, adjust loads, and add load-shedding generators.

In [4]:
n_elec_path = "data/networks/base_s_50_elec_.nc"
solver_name = "gurobi"
solver_options = {"OutputFlag": 0}  # minimize solver log output

# Load network
n = pypsa.Network(n_elec_path)

# Base styling and carriers
n.carriers.loc["", "color"] = "#aaaaaa"
n.add("Carrier", "load shedding", color="#aa0000")

# Increase load
n.loads_t.p_set *= 1.5

# Add load-shedding generators
spatial_nodes = n.buses.index.tolist()
ls_names = [f"{bus} load shedding" for bus in spatial_nodes]

n.add(
    "Generator",
    pd.Index(ls_names),
    bus=pd.Series(spatial_nodes, index=ls_names),
    p_nom_extendable=True,
    p_nom_max=np.inf,
    capital_cost=0.1,
    marginal_cost=10000.0,
    carrier="load shedding",
)

INFO:pypsa.network.io:New version 1.0.5 available! (Current: 1.0.4)
INFO:pypsa.network.io:Imported network 'Unnamed Network' has buses, carriers, generators, lines, links, loads, storage_units, stores, sub_networks


## Optimisation
Prepare scenario copies for baseline and islanded configurations. Run optimisations for baseline, s_max_pu = 0 / p_max_pu = 0, and fully islanded cases.

In [5]:
print("Running baseline optimisation...")
n1 = n.copy()
n1.optimize(solver_name=solver_name, solver_options=solver_options)

cap1 = extract_capacity(n1)
eb1 = extract_energy_balance(n1)

Running baseline optimisation...


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 14/14 [00:00<00:00, 551.18it/s]
INFO:linopy.io: Writing time: 0.33s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


...

Reading time = 0.11 seconds


INFO:gurobipy:Reading time = 0.11 seconds


obj: 53897 rows, 25873 columns, 107777 nonzeros


INFO:gurobipy:obj: 53897 rows, 25873 columns, 107777 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 25873 primals, 53897 duals
Objective: 7.66e+10
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, Kirchhoff-Voltage-Law, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


In [6]:
print("Running islanded optimisation (s_max_pu = 0 and p_max_pu = 0)...")
n2 = n.copy()
if not n.lines.empty:
    n2.lines["s_max_pu"] = 0
if not n2.links.empty:
    dc_links = n2.links[n2.links.carrier == "DC"].index
    n2.links.loc[dc_links, ["p_max_pu", "p_min_pu"]] = 0

n2.optimize(solver_name=solver_name, solver_options=solver_options)

cap2 = extract_capacity(n2)
eb2 = extract_energy_balance(n2)

Running islanded optimisation (s_max_pu = 0 and p_max_pu = 0)...


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 14/14 [00:00<00:00, 565.53it/s]
INFO:linopy.io: Writing time: 0.3s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


...

Reading time = 0.07 seconds


INFO:gurobipy:Reading time = 0.07 seconds


obj: 53897 rows, 25873 columns, 100769 nonzeros


INFO:gurobipy:obj: 53897 rows, 25873 columns, 100769 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 25873 primals, 53897 duals
Objective: 9.80e+10
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, Kirchhoff-Voltage-Law, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


In [7]:
print("Running islanded optimisation (lines and links removed)...")
n3 = n.copy()
if not n.lines.empty:
    n3.remove("Line", n3.lines.index)
if not n3.links.empty:
    n3.remove("Link", n3.links[n3.links.carrier == "DC"].index)

n3.optimize(solver_name=solver_name, solver_options=solver_options)

cap3 = extract_capacity(n3)
eb3 = extract_energy_balance(n3)

Running islanded optimisation (lines and links removed)...


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 12/12 [00:00<00:00, 436.03it/s]
INFO:linopy.io: Writing time: 0.24s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


...

Reading time = 0.06 seconds


INFO:gurobipy:Reading time = 0.06 seconds


obj: 44499 rows, 21798 columns, 80475 nonzeros


INFO:gurobipy:obj: 44499 rows, 21798 columns, 80475 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 21798 primals, 44499 duals
Objective: 9.80e+10
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


In [8]:
print("Running islanded optimisation (AC lines and DC links zero availability in summer: Q2+Q3)...")
n4 = n.copy()

snapshots = n4.snapshots
quarters = np.array_split(snapshots, 4)
q1, q2, q3, q4 = quarters
outage_summer = q2.union(q3)

if not n4.lines.empty:
    lines_avail = pd.DataFrame(1.0, index=snapshots, columns=n4.lines.index)
    lines_avail.loc[outage_summer, :] = 0.0
    n4.lines_t.s_max_pu = lines_avail

if not n4.links.empty:
    dc_links = n4.links.index[n4.links.carrier == "DC"]
    links_avail = pd.DataFrame(1.0, index=snapshots, columns=n4.links.index)
    links_avail.loc[outage_summer, dc_links] = 0.0
    n4.links_t.p_max_pu = links_avail
    n4.links_t.p_min_pu = -links_avail

n4.optimize(solver_name=solver_name, solver_options=solver_options)

cap4 = extract_capacity(n4)
eb4 = extract_energy_balance(n4)

Running islanded optimisation (AC lines and DC links zero availability in summer: Q2+Q3)...


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 14/14 [00:00<00:00, 483.36it/s]
INFO:linopy.io: Writing time: 0.3s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


...

Reading time = 0.09 seconds


INFO:gurobipy:Reading time = 0.09 seconds


obj: 53897 rows, 25873 columns, 107081 nonzeros


INFO:gurobipy:obj: 53897 rows, 25873 columns, 107081 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 25873 primals, 53897 duals
Objective: 8.26e+10
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, Kirchhoff-Voltage-Law, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


In [9]:
print("Running islanded optimisation (AC lines and DC links zero availability in winter: Q1+Q4)...")
n5 = n.copy()

snapshots = n5.snapshots
quarters = np.array_split(snapshots, 4)
q1, q2, q3, q4 = quarters
outage_winter = q1.union(q4)

if not n5.lines.empty:
    lines_avail_2 = pd.DataFrame(1.0, index=snapshots, columns=n5.lines.index)
    lines_avail_2.loc[outage_winter, :] = 0.0
    n5.lines_t.s_max_pu = lines_avail_2

if not n5.links.empty:
    dc_links = n5.links.index[n5.links.carrier == "DC"]
    links_avail_2 = pd.DataFrame(1.0, index=snapshots, columns=n5.links.index)
    links_avail_2.loc[outage_winter, dc_links] = 0.0
    n5.links_t.p_max_pu = links_avail_2
    n5.links_t.p_min_pu = -links_avail_2

n5.optimize(solver_name=solver_name, solver_options=solver_options)

cap5 = extract_capacity(n5)
eb5 = extract_energy_balance(n5)

Running islanded optimisation (AC lines and DC links zero availability in winter: Q1+Q4)...


INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 14/14 [00:00<00:00, 549.38it/s]
INFO:linopy.io: Writing time: 0.37s


Set parameter WLSAccessID


INFO:gurobipy:Set parameter WLSAccessID


Set parameter WLSSecret


...

Reading time = 0.07 seconds


INFO:gurobipy:Reading time = 0.07 seconds


obj: 53897 rows, 25873 columns, 107081 nonzeros


INFO:gurobipy:obj: 53897 rows, 25873 columns, 107081 nonzeros


INFO:gurobipy:Warning: environment still referenced so free is deferred (Continue to use WLS)
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 25873 primals, 53897 duals
Objective: 8.54e+10
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Line-ext-s-lower, Line-ext-s-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, Kirchhoff-Voltage-Law, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


## Metrics
Metrics for validation

In [10]:
total_costs = pd.DataFrame({
    "scenario": [
        "Grid exists",
        "Grid outage (full year, s_max_pu=0)",
        "Grid outage (full year, lines removed)",
        "Grid outage (Summer)",
        "Grid outage (Winter)",
    ],
    "total_cost": [
        np.round(calculate_total_costs(n1) / 1e9, 3),
        np.round(calculate_total_costs(n2, exclude_grid=True) / 1e9, 3),
        np.round(calculate_total_costs(n3, exclude_grid=True) / 1e9, 3),
        np.round(calculate_total_costs(n4, exclude_grid=False) / 1e9, 3),
        np.round(calculate_total_costs(n5, exclude_grid=False) / 1e9, 3),
    ],
})
print("\nTotal annual costs by scenario:")
print(total_costs.to_string(index=False))


Total annual costs by scenario:
                              scenario  total_cost
                           Grid exists     304.914
   Grid outage (full year, s_max_pu=0)     317.299
Grid outage (full year, lines removed)     317.299
                  Grid outage (Summer)     310.923
                  Grid outage (Winter)     313.674


In [11]:
# mp_diff = n3.buses_t.marginal_price - n2.buses_t.marginal_price
# mp_diff_mean = mp_diff.abs().mean()
# print("\nMean absolute marginal price difference:")
# print(mp_diff_mean.head())

## Plots
Optimal capacities and energy balances across scenarios.

In [13]:
capacities = pd.concat([
    cap1.assign(scenario="grid"),
    cap2.assign(scenario="maxpu0"),
    cap3.assign(scenario="removed"),
    cap4.assign(scenario="outage_summer"),
    cap5.assign(scenario="outage_winter"),
], ignore_index=True)

# Convert capacity from MW to GW for better readability
capacities["capacity_GW"] = capacities["capacity"] / 1e3

capacities_pivot = capacities.pivot_table(
    index=["component", "name"],
    columns="scenario",
    values="capacity_GW",
    fill_value=0,
).reset_index()

# Map name to carrier and color
name_to_carrier = pd.concat([
    n.generators["carrier"],
    n.storage_units["carrier"],
    n.stores["carrier"]
])

fig_cap = create_bar_plot(
    capacities_pivot,
    "name",
    ["grid", "maxpu0", "removed", "outage_summer", "outage_winter"],
    [
        "Grid exists",
        "Grid outage (full year, s_max_pu=0)",
        "Grid outage (full year, lines removed)",
        "Grid outage (Summer)",
        "Grid outage (Winter)",
    ],
    "Optimal capacity",
    "Installed capacity (GW)",
    height=600,
    width=1600,
)
fig_cap.show()

In [14]:
energy_balances = pd.concat([
    eb1.assign(scenario="grid"),
    eb2.assign(scenario="maxpu0"),
    eb3.assign(scenario="removed"),
    eb4.assign(scenario="outage_summer"),
    eb5.assign(scenario="outage_winter"),
], ignore_index=True)

# Convert energy from MWh to TWh for better readability
energy_balances["energy_TWh"] = energy_balances["energy"] / 1e6

# Map carriers to colors from n.carriers
carrier_color_map = {carrier: n.carriers.loc[carrier, "color"] 
                     for carrier in energy_balances["carrier"].unique() 
                     if carrier in n.carriers.index}

# Define scenario labels
scenario_labels = {
    "grid": "Grid exists",
    "maxpu0": "Grid outage (full year, s_max_pu=0)",
    "removed": "Grid outage (full year, lines removed)",
    "outage_summer": "Grid outage (Summer)",
    "outage_winter": "Grid outage (Winter)",
}

fig_eb_stacked = create_stacked_bar_plot(
    energy_balances,
    scenario_col="scenario",
    value_col="energy_TWh",
    carrier_col="carrier",
    carrier_colors=carrier_color_map,
    scenario_labels=scenario_labels,
    title="Energy balance by scenario",
    yaxis_title="Energy (TWh)",
    height=800,
    width=800,
)
fig_eb_stacked.show()